In [ ]:
from urllib.request import urlretrieve
import pandas as pd

In [ ]:
italy_covid_url = 'https://gist.githubusercontent.com/aakashns/f6a004fa20c84fec53262f9a8bfee775/raw/f309558b1cf5103424cef58e2ecb8704dcd4d74c/italy-covid-daywise.csv'

covid_df = pd.read_csv(italy_covid_url)
covid_df = pd.DataFrame(covid_df)
print(covid_df)

In [ ]:
# prints out top five rows by default
covid_df.head(10)

In [ ]:
# prints out last five rows by default
covid_df.tail(10)

In [ ]:
# Object is a generic data type when panda can not figure out the data type specified
covid_df.info()

covid_df.describe()

covid_df.columns

covid_df.shape


In [ ]:
# Retrieve data from dataframe
# Internal representation of a data frame
# A dictionary or list

# Pandas format is simliar to this
covid_data_dict = {
    'date':       ['2020-08-30', '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03'],
    'new_cases':  [1444, 1365, 996, 975, 1326],
    'new_deaths': [1, 4, 6, 8, 6],
    'new_tests': [53541, 42583, 54395, None, None]
}

print(covid_data_dict)

In [ ]:
# Access data based on row range
# To access a specific row of data, Pandas provides the `.loc` method.
covid_df.loc[1:2]
# Access columns based on labels
# Further, you can also pass a list of columns within the indexing notation `[]` 
# to access a subset of the data frame with just the given columns.
covid_df[['new_cases','new_deaths']]
# Access at specified index
covid_df.at[200, 'new_tests']

In [ ]:
# What are the total number of reported cases and deaths related to Covid-19 in Italy?
total_cases = covid_df.new_cases.sum()
total_deaths = covid_df.new_deaths.sum()
print(total_cases)
print(total_deaths)

In [ ]:
# What is the overall death rate (ratio of reported deaths to reported cases)?
death_rate = covid_df.new_deaths.sum() / covid_df.new_cases.sum()
print(death_rate)

In [ ]:
# What is the overall number of tests conducted? A total of 935310 tests were conducted before daily 
# test numbers were reported.

initial_tests = 935310
total_tests = initial_tests + covid_df.new_tests.sum()
print(total_tests)

In [ ]:
# What fraction of tests returned a positive result?
positive_rate = total_cases / total_tests
print('{:.2f}% of tests in Italy led to a positive diagnosis.'.format(positive_rate*100))

In [ ]:
# Querying and sorting rows
high_new_cases = covid_df.new_cases > 1000
# The boolean expression returns a series containing `True` and `False` boolean values. 
# You can use this series to select a subset of rows from the original dataframe, corresponding to the 
# `True` values in the series.
covid_df[high_new_cases]

# We can write this succinctly on a single line by passing the boolean expression as an index to the data frame.
high_cases_df = covid_df[covid_df.new_cases > 1000]
high_cases_df

In [ ]:
locate = covid_df.loc[10:50:2] # For locating rows. start:end:increament
locate

In [ ]:
covid_df.loc[10:20,['date','new_cases']] # This will print out rows and columns

In [ ]:
covid_df.iloc[2:7,3:5] # First set of argument is for rows and second is for columns

In [ ]:
covid_df.query('date == ["2020-08-21","2020-08-23","2020-08-24","2020-08-25","2020-08-26","2020-08-28"] & (new_cases>2)')

In [ ]:
# Compare two sequences
newDeaths = covid_df['new_deaths']

covid_df['new_cases'].isin(newDeaths)

In [ ]:
# Let's try to determine the days when the ratio 
# of cases reported to tests conducted is higher than the overall positive_rate
high_ratio_df = covid_df[covid_df.new_cases / covid_df.new_tests > positive_rate]
print(high_ratio_df)

In [ ]:
# We can use this series to add a new column to the data frame.
covid_df['positive_rate'] = covid_df.new_cases / covid_df.new_tests
covid_df

In [ ]:
# For now, let's remove the positive_rate column using the drop method.
covid_df.drop(columns=['positive_rate'], inplace=True)
covid_df

In [ ]:
# Sorting rows using column values
# The rows can also be sorted by a specific column using .sort_values. Let's sort to identify the days with 
# the highest number of cases, then chain it with the head method to list just the first ten results.

covid_df.sort_values('new_cases', ascending=False).head(10)

In [ ]:
# Working with dates
covid_df.date
covid_df['date'] = pd.to_datetime(covid_df.date)
covid_df['date']

In [ ]:
# You can see that it now has the datatype datetime64. We can now extract different parts of the data into 
# separate columns, using the DatetimeIndex 

covid_df['year'] = pd.DatetimeIndex(covid_df.date).year
covid_df['month'] = pd.DatetimeIndex(covid_df.date).month
covid_df['day'] = pd.DatetimeIndex(covid_df.date).day
covid_df['weekday'] = pd.DatetimeIndex(covid_df.date).weekday
covid_df

In [ ]:
# Let's check the overall metrics for May. We can query the rows for May, choose a subset of columns, and use the 
# `sum` method to aggregate each selected column's values.

# Query the rows for May
covid_df_may = covid_df[covid_df.month == 5]

# Extract the subset of columns to be aggregated
covid_df_may_metrics = covid_df_may[['new_cases', 'new_deaths', 'new_tests']]

# Get the column-wise sum
covid_may_totals = covid_df_may_metrics.sum()
covid_may_totals

In [ ]:
# As another example, let's check if the number of cases reported on Sundays is higher than the average number of cases reported every day. 
# This time, we might want to aggregate columns using the `.mean` method.

# Overall average
covid_df.new_cases.mean()

In [ ]:
# Apply function

def newDeaths(x):
    if x > 5:
        return x
    else:
        return None
results = covid_df['new_deaths'].apply(newDeaths)
results

In [ ]:
# Grouping and aggregation
# As a next step, we might want to summarize the day-wise data and create a new dataframe with month-wise data. 
# We can use the `groupby` function to create a group for each month, select the columns we wish to aggregate, 
# and aggregate them using the `sum` method. 

# covid_month_df = covid_df.groupby('month')[['new_cases', 'new_deaths', 'new_tests']].sum()
# covid_month_df = covid_df.groupby('month')[['new_cases', 'new_deaths', 'new_tests']].mean()
# covid_month_df = covid_df.groupby('month')[['new_cases', 'new_deaths', 'new_tests']].median()
covid_month_df = covid_df.groupby('month')[['new_cases', 'new_deaths', 'new_tests']].std()
covid_month_df

In [ ]:
covid_weekday_df = covid_df.groupby('weekday')[['new_cases', 'new_deaths', 'new_tests']].sum()
covid_weekday_df

In [ ]:
covid_day_df = covid_df.groupby('day')[['new_cases', 'new_deaths', 'new_tests']].sum()
covid_day_df

In [ ]:
# Apart from grouping, another form of aggregation is the running or cumulative sum of cases, tests, or death up to 
# each row's date. We can use the cumsum method to compute the cumulative sum of a column as a new series. Let's add 
# three new columns: total_cases, total_deaths, and total_tests.

covid_df['total_cases'] = covid_df.new_cases.cumsum()
covid_df['total_deaths'] = covid_df.new_deaths.cumsum()
covid_df['total_tests'] = covid_df.new_tests.cumsum() + initial_tests
covid_df

In [1]:
# Merging data from multiple sources
# To determine other metrics like test per million, cases per million, etc., we require some more information about the 
# country, viz. its population. Let's download another file locations.csv that contains health-related information for 
# many countries, including Italy.

urlretrieve('https://gist.githubusercontent.com/aakashns/8684589ef4f266116cdce023377fc9c8/raw/99ce3826b2a9d1e6d0bde7e9e559fc8b6e9ac88b/locations.csv', 
            'data/locations.csv')

locations_df = pd.read_csv('data/locations.csv')
locations_df

NameError: name 'urlretrieve' is not defined

In [ ]:
locations_df[locations_df.location == "Italy"]
# We can merge this data into our existing data frame by adding more columns. However, to merge two data frames, 
# we need at least one common column. Let's insert a location column in the covid_df dataframe with all values set to 
# "Italy".
covid_df['location'] = "Italy"
covid_df

# We can now add the columns from `locations_df` into `covid_df` using the `.merge` method.
merged_df = covid_df.merge(locations_df, on="location")
merged_df

In [ ]:
# The location data for Italy is appended to each row within `covid_df`. If the `covid_df` data frame contained data for 
# multiple locations, then the respective country's location data would be appended for each row.
# We can now calculate metrics like cases per million, deaths per million, and tests per million.

merged_df['cases_per_million'] = merged_df.total_cases * 1e6 / merged_df.population
merged_df['deaths_per_million'] = merged_df.total_deaths * 1e6 / merged_df.population
merged_df['tests_per_million'] = merged_df.total_tests * 1e6 / merged_df.population
merged_df

In [ ]:
# Writing data back to files
# After completing your analysis and adding new columns, you should write the results back to a file. Otherwise, 
# the data will be lost when the Jupyter notebook shuts down. Before writing to file, let us first create a data frame 
# containing just the columns we wish to record.

result_df = merged_df[['date',
                       'new_cases', 
                       'total_cases', 
                       'new_deaths', 
                       'total_deaths', 
                       'new_tests', 
                       'total_tests', 
                       'cases_per_million', 
                       'deaths_per_million', 
                       'tests_per_million']]

result_df

In [ ]:
# To write the data from the data frame into a file, we can use the `to_csv` function. 
result_df.to_csv('data/results.csv', index=None)

In [ ]:
# Basic Plotting with Pandas
# We generally use a library like matplotlib or seaborn plot graphs within a Jupyter notebook. However, Pandas dataframes
#  & series provide a handy .plot method for quick and easy plotting.
result_df.new_cases.plot()


In [ ]:
# While this plot shows the overall trend, it's hard to tell where the peak occurred, as there are no dates on the X-axis. 
# We can use the `date` column as the index for the data frame to address this issue.

result_df.set_index('date', inplace=True)
result_df

In [ ]:
# Notice that the index of a data frame doesn't have to be numeric. Using the date as the index also allows us
#  to get the data for a specific data using `.loc`.

result_df.loc['2020-09-01']

In [ ]:
# Let's plot the new cases & new deaths per day as line graphs.
result_df.new_cases.plot()
result_df.new_deaths.plot()

In [ ]:
# We can also compare the total cases vs. total deaths.
result_df.total_cases.plot()
result_df.total_deaths.plot()

In [ ]:
# Let's see how the death rate and positive testing rates vary over time.
death_rate = result_df.total_deaths / result_df.total_cases
death_rate.plot(title='Death Rate')

In [ ]:
positive_rates = result_df.total_cases / result_df.total_tests
positive_rates.plot(title='Positive Rate')

In [ ]:
# Finally, let's plot some month-wise data using a bar chart to visualize the trend at a higher level.
covid_month_df.new_cases.plot(kind='bar')

In [ ]:
covid_month_df.new_tests.plot(kind='bar')

In [ ]:
covid_df

In [ ]:
covid_df.dropna() # This will drop all the rows with no data

In [ ]:
# drop any columns with missing values
# Column new_tests is dropped
covid_df.dropna(axis='columns',how='any')


In [ ]:
# Drop with respect to condition given
# From index, drop all rows were total_tests == NAN
covid_df.dropna(axis='index',how='any',subset=['total_tests'])

In [141]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        248 non-null    object 
 1   new_cases   248 non-null    int32  
 2   new_deaths  248 non-null    float64
 3   new_tests   135 non-null    float64
dtypes: float64(2), int32(1), object(1)
memory usage: 6.9+ KB


In [142]:
covid_df = covid_df.astype({'new_cases':int})
covid_df = covid_df.astype({'new_deaths':str})
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        248 non-null    object 
 1   new_cases   248 non-null    int32  
 2   new_deaths  248 non-null    object 
 3   new_tests   135 non-null    float64
dtypes: float64(1), int32(1), object(2)
memory usage: 6.9+ KB
